# Using Remote Services (  Programming ) 
While applications that use Remote Services can generally be built without having to consider
where they will be run, there are a few aspects of Remote Services that programmers should be
aware of. These are covered in this section.

## Using an API to Create a Compute Server Job

When remote resources are tied to the optimization process, such as a Cluster Manager, compute server, or Instant Cloud, extra care should be taken to ensure that such resources are released once they are no longer needed. Technically, such resources are managed by a gurobipy.Env object ("environment").  This example shows best practices for acquiring and releasing such shared resources via Env objects.

See also 

https://www.gurobi.com/documentation/9.1/refman/environments.html

https://www.gurobi.com/documentation/10.0/examples/mip1_remote_py.html

## Using With Statmenet

In [3]:
import gurobipy as gp
from gurobipy import GRB


def populate_and_solve(m):
    # This function formulates and solves the following MIP model (see mip1.py):
    #  maximize
    #        x +   y + 2 z
    #  subject to
    #        x + 2 y + 3 z <= 4
    #        x +   y       >= 1
    #        x, y, z binary

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

    print('Obj: %g' % m.ObjVal)

# Put any connection parameters for Gurobi Compute Server, Gurobi Cluster
# Manager or Gurobi Token server here, unless they are set already
# through the license file.

connection_params = {
    "CSManager": "https://demos-manager.gurobi.com",
    "CSAPIAccessID": "e41c4ff4-4322-4218-a52b-f82047083523",
    "CSAPISecret": "e62cc65e-2b26-4726-8a2a-2a2fe77157ec", 
    "CSAPPNAME": "Compass"
    
# For Compute Server you need at least this
#       "ComputeServer": "<server name>",
#       "UserName": "<user name>",
#       "ServerPassword": "<password>",

# For Cluster Manager you need at least this
#       "CSManager": "<manager name>",
#       "CSAPIAccessID": "<access ID>",
#       "CSAPISecret": "<secret>",

# For Instant cloud you need at least this
#       "CloudAccessID": "<access id>",
#       "CloudSecretKey": "<secret>",
        }

with gp.Env(params=connection_params) as env:
    # 'env' is now set up according to the connection parameters.
    # The environment is disposed of automatically through the context manager
    # upon leaving this block.
    with gp.Model(env=env) as model:
        # 'model' is now an instance tied to the enclosing Env object 'env'.
        # The model is disposed of automatically through the context manager
        # upon leaving this block.
        try:
            populate_and_solve(model)
        except:
            # Add appropriate error handling here.
            raise

Set parameter CSManager to value "https://demos-manager.gurobi.com"
Set parameter CSAPIAccessID
Set parameter CSAPISecret
Set parameter CSAppName to value "Compass"
Compute Server job ID: 57fc1ac0-1144-449e-a735-02cc0497970b
Capacity available on '192.168.60.154:61000' - connecting...
Established HTTPS encrypted connection
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)
Gurobi Compute Server Worker version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) Platinum 8275CL CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows a

## Using env.dispose
When you are a client of a Gurobi Compute Server, starting an environment will start a job on the server (or enter the queue if the server is fully occupied). Disposing of the environment will end that job, allowing the next job in the queue to start. 

https://www.gurobi.com/documentation/9.5/refman/session_boundaries.html

In [5]:
import gurobipy as gp
from gurobipy import GRB

env= gp.Env(empty=True)
env.setParam('CSMANAGER',"https://demos-manager.gurobi.com")
env.setParam('CSAPIACCESSID',"e41c4ff4-4322-4218-a52b-f82047083523")
env.setParam('CSAPISECRET',"e62cc65e-2b26-4726-8a2a-2a2fe77157ec")
env.setParam('CSAPPNAME',"Compass")
env.start()


# This function formulates and solves the following MIP model (see mip1.py):
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary
# Create a new model
m = gp.Model("mip1",env)
# Create variables
x = m.addVar(vtype=GRB.BINARY, name="x")
y = m.addVar(vtype=GRB.BINARY, name="y")
z = m.addVar(vtype=GRB.BINARY, name="z")

# Set objective
m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

# Add constraint: x + 2 y + 3 z <= 4
m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

# Add constraint: x + y >= 1
m.addConstr(x + y >= 1, "c1")

# Optimize model
m.optimize()

for v in m.getVars():
  print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)

m.dispose()
env.dispose()


Set parameter CSManager to value "https://demos-manager.gurobi.com"
Set parameter CSAPIAccessID
Set parameter CSAPISecret
Set parameter CSAppName to value "Compass"
Compute Server job ID: 29ed3f0c-e5dd-4c1e-9ad8-3898ebddb586
Capacity available on '192.168.3.117:61000' - connecting...
Established HTTPS encrypted connection
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)
Gurobi Compute Server Worker version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) Platinum 8275CL CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows an